# GeoClimate-Fetcher Interactive GUI

This notebook provides an interactive interface for downloading climate data from Google Earth Engine.

## Workflow

1. Authenticate with Google Earth Engine
2. Select an area of interest
3. Choose a dataset and bands
4. Set time range
5. Configure and run the download

In [1]:
# Import required libraries
import os
import sys
import ipywidgets as widgets
from IPython.display import display, clear_output
import ee
from datetime import datetime, date
from pathlib import Path
import geemap

# Import GeoClimate-Fetcher modules
from geoclimate_fetcher.core import (
    authenticate,
    MetadataCatalog,
    GeometryHandler,
    GEEExporter, 
    ImageCollectionFetcher,
    StaticRasterFetcher
)

from geoclimate_fetcher.ui import (
    AuthWidget, 
    MapWidget,
    DatasetPickerWidget,
    BandPickerWidget, 
    TimeSliderWidget,
    DownloadDialogWidget
)

ModuleNotFoundError: No module named 'geoclimate_fetcher'

## 0. Setup and Initialization

Initialize necessary global variables and objects.

In [ ]:
# Create core objects
metadata_catalog = MetadataCatalog()
geometry_handler = GeometryHandler()
exporter = GEEExporter()

# Initialize state variables
auth_complete = False
geometry_complete = False
dataset_selected = False
bands_selected = False
dates_selected = False

# Data parameters
current_dataset = None
selected_bands = []
start_date = None
end_date = None
snippet_type = None
download_path = None

## 1. Authentication

First, you must authenticate with Google Earth Engine. Enter your GEE Project ID to continue.

In [ ]:
# Callback for successful authentication
def on_auth_success(auth):
    global auth_complete
    auth_complete = True
    show_main_workflow()

# Show the main workflow after authentication
def show_main_workflow():
    if not auth_complete:
        print("Please authenticate first.")
        return
    # Show the AOI selection widget
    clear_output()
    print("Authentication successful. Please select an area of interest.")
    map_widget.display()

# Initialize the authentication widget
auth_widget = AuthWidget(on_auth_success=on_auth_success)
auth_widget.display()

## 2. Area of Interest

After authentication, use the map to draw an area of interest or import a GeoJSON/Shapefile.

In [ ]:
# Callback for geometry selection
def on_geometry_selected(geometry_handler):
    global geometry_complete
    geometry_complete = True
    print("Geometry selected. Now you can select a dataset.")
    dataset_picker.display()

# Initialize map widget
map_widget = MapWidget(on_geometry_selected=on_geometry_selected)

# Don't display yet - wait for authentication

## 3. Dataset and Band Selection

After selecting an area, you can choose a dataset and the specific bands you want to download.

In [ ]:
# Callback for dataset selection
def on_dataset_selected(dataset):
    global dataset_selected, current_dataset, snippet_type
    dataset_selected = True
    current_dataset = dataset
    snippet_type = dataset.get('Snippet Type')
    # Update band picker and time slider with the selected dataset
    band_picker.set_dataset(dataset)
    time_slider.set_dataset(dataset)
    print(f'Selected dataset: {dataset.get("Dataset Name")} ({snippet_type})')
    print("Please select the bands you want to download.")
    band_picker.display()

# Callback for band selection
def on_bands_selected(bands):
    global bands_selected, selected_bands
    bands_selected = True
    selected_bands = bands
    print(f'Selected bands: {, join(bands)}')
    print("Now please select the time range.")
    # Show time slider for ImageCollections, or the download dialog for single Images
    if snippet_type == 'ImageCollection':
        time_slider.display()
    else:
        initialize_download_dialog()

# Initialize dataset picker widget
dataset_picker = DatasetPickerWidget(metadata_catalog, on_dataset_selected=on_dataset_selected)
# Initialize band picker widget
band_picker = BandPickerWidget(metadata_catalog, on_bands_selected=on_bands_selected)

# Don't display yet - wait for geometry selection

## 4. Time Range Selection

For ImageCollections, select the time range for which you want to download data.

In [ ]:
# Callback for date range selection
def on_dates_selected(start_d, end_d):
    global dates_selected, start_date, end_date
    dates_selected = True
    start_date = start_d
    end_date = end_d
    print(f'Selected time range: {start_date} to {end_date}')
    print("Now you can configure the download.")
    initialize_download_dialog()

# Initialize time slider widget
time_slider = TimeSliderWidget(metadata_catalog, on_dates_selected=on_dates_selected)

# Don't display yet - wait for band selection

## 5. Download Configuration and Execution

Finally, configure the download parameters and start the download process.

In [ ]:
# Callback for download completion
def on_download_complete():
    print("Download complete!")
    print(f'File saved to: {download_path}')

# Initialize download dialog
def initialize_download_dialog():
    global download_dialog
    # Initialize the download dialog
    download_dialog = DownloadDialogWidget(
        metadata_catalog, 
        geometry_handler, 
        on_download_complete=on_download_complete
    )` 
    
    # Set parameters based on previous selections
    dataset_name = current_dataset.get('Dataset Name')
    ee_id = current_dataset.get('Earth Engine ID')
    download_dialog.set_parameters(
        dataset_name, 
        ee_id, 
        selected_bands, 
        start_date, 
        end_date
    )
    
    # Display the download dialog
    download_dialog.display()

# Implemented on-demand in previous cells

## Example: Downloading ERA5 Precipitation Data

Follow these steps to download ERA5 precipitation data:

1. First, authenticate with your GEE Project ID
2. Draw a polygon on the map and click 'Save Selection'
3. Select 'precipitation' from the category dropdown and choose 'ERA5 Daily Aggregates'
4. Select the 'total_precipitation' band
5. Choose a date range (e.g., the last month)
6. Set extraction mode to 'Average time-series' and click 'Start Download'

## Additional Usage Notes

- For static rasters (e.g., DEM), you can skip the time range selection
- You can export data directly to your local disk or to Google Drive
- For large areas or long time series, use 'Export to Google Drive' 
- The 'Gridded rasters' extraction mode provides pixel-level data